# Data acquisition from .html files

This notebook acquires data from the dataset1

Requirements:

*   Beautiful Soup - Removes html tags.
*   Pandas - Data manipulation.

The next cells are needed to run the notebook inside the Google Colaboratory platform, with the datasets in Google Drive

In [ ]:
!pip install dateparser

Setting up connection to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# dataset1 location in Google Drive
CSV_falencias = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados/falencias.csv'
CSV_recjud = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados/recuperacoes_judiciais.csv'
PATH_save = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados'


The next cell is needed to run the notebook locally, with the datasets available in a local disk

In [18]:
#CSV_falencias = '/home/raktanaka/USP/dataset/dataset1/falencias.csv'
#CSV_unzip = '/home/raktanaka/USP/dataset/dataset1/recuperacoes_judiciais.csv'
CSV_falencias = '/run/media/raktanaka/KINGSTON/dataset1/falencias.csv'
CSV_recjud = '/run/media/raktanaka/KINGSTON/dataset1/recuperacoes_judiciais.csv'
PATH_save = '/home/raktanaka/USP/MAC6967-G6-NLP-juridico'

In [17]:
import os
import csv
import pandas as pd
from bs4 import BeautifulSoup

# Increase CSV allocation size
csv.field_size_limit(sys.maxsize)

9223372036854775807

2 Functions for data extraction

In [19]:
#Cleaning
def Cleaning(df):

    for i in range(0,len(df)):
        
        soup = BeautifulSoup(df['html'][i])

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        text = "".join(text.split('\\n'))
        text = "".join(text.split('\\t')).replace('\\n',' ')
        #text = "".join(text.split('\n'))
        text = text.replace('\\xa0','')
        text = text.replace('Advogada','Advogado')
        df['html'][i] = text.lower()

    return(df)


In [20]:
def GetData(df):

  #New dataframe
  dfinal = pd.DataFrame(columns=['num_proc','status','juiz','terceiros','rep_legal','valor_acao','reqte','adv_reqte','reqdo','adv_reqdo'])
  errors=[]
  for i in range(0,len(df)):
    #print(i)
    try:
    #Numero processo e satus
      a="".join(re.findall("[\d.-]", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('classe:')[0]))
      #s="".join(re.findall("[^\d\W]", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('classe:')[0]))
      s="".join(re.findall("[^\d:.,-]+", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('classe:')[0]))
    except:
      a=''
      s=''
      errors+=[i]

    try:

      t=df['html'][i].split('partes do processo')[1].split('movimentações')[0].split('terintcer:')[1].split('\n')[0]
      r=",".join(list(filter(lambda x: 'repreleg:' in x,df['html'][i].split('reqdo')[1].split('\n'))))
      r=r.replace('repreleg:','')
    except:
      t=r=''

    try:
      #Juiz, terceiros, representante
      #b=df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0].replace('\n','')
      if 'outros' in df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0]:
        b=" ".join(re.findall(r'[^\s0-9-,.]+',df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0])[:-2])
      else:
        b=" ".join(re.findall(r'[^\s0-9-,.]+',df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0]))

    except:
      b=''
      errors+=[i]
    #Valor da ação
    try:
      c=re.findall("r[$]\d{1,3}.\d{3}\,\d{2}", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('valor da ação:')[1])[0]
      c=c.replace('r$','')
      #c=re.findall("[r$]\d{1,3}.\d{3}\,\d{2}", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('valor da ação:')[1])[0]
    except:
      c=''
      errors+=[i]
    try:
      if 'reqdo' in df['html'][i]:
        x='reqdo'
      else:
        x='falido'
      #Requerente
      d=" ".join(re.findall(r'[^\s0-9-,.]+',df['html'][i].split('reqte:')[1].split('advogado')[0]))
      #d=df['html'][i].split('reqte:')[1].split('advogado')[0]
      #Advogado Requerente
      e=" ".join(re.findall(r'[^\s0-9-,.:]+',df['html'][i].split('reqte:')[1].split('advogado')[1].split(x)[0]))
      #e=df['html'][i].split('reqte:')[1].split('advogado')[1].split(x)[0]
      #Requerido
      #f=" ".join(re.findall(r'[^\s0-9-,.:]+',df['html'][i].split(x)[1].split('advogado')[0]))
      #f=df['html'][i].split(x)[1].split('advogado')[0]
      f=re.findall(r'[^0-9-,.:]+',df['html'][i].split(x)[1].split('\n')[0])[0]
      #Advogado Requerido
      g=",".join(list(filter(lambda x: 'advogado:' in x,df['html'][i].split(x)[1].split('\n'))))
      g=g.replace('advogado:','')
    
    except:
      errors+=[i]
      d=e=f=g=''

    dfinal=dfinal.append({'num_proc':a,'status':s,'juiz':b,'terceiros':t,'rep_legal':r,'valor_acao':c,'reqte':d,'adv_reqte':e,'reqdo':f,'adv_reqdo':g},ignore_index=True)
    errors=list(set(errors))
  return(dfinal)


The next 2 cells execute the data extraction

In [21]:
#Main data - falencias
DF_save = 'falencias_html.csv'

df = pd.read_csv(CSV_falencias)
df['NumeroDoProcesso']=df['NumeroDoProcesso'].str.replace(']','').str.replace('[','')
df['html']=df['html'].str.replace(']','').str.replace('[','')

df_falencias = Cleaning(df)
df_falencias = GetData(df_falencias)

save = os.path.join(PATH_save, DF_save)
df_falencias.to_csv(save, index = False)


In [23]:
#Main data - recuperacoes_judiciais
DF_save = 'recuperacoes_judiciais_html.csv'

df = pd.read_csv(CSV_recjud)
df['NumeroDoProcesso']=df['NumeroDoProcesso'].str.replace(']','').str.replace('[','')
df['html']=df['html'].str.replace(']','').str.replace('[','')

df_recjud = Cleaning(df)
df_recjud = GetData(df_recjud)

save = os.path.join(PATH_save, DF_save)
df_recjud.to_csv(save, index = False)